In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

# Pandas Indexing and Selecting

## Review the Basics

Pertama, mari kita mulai dengan sedikit rekap tentang indexing dan pemilihan selection. Untuk memulainya, mari kita gunakan dataset tips:

In [2]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


Pada dasarnya ada 4 cara untuk mendapatkan data dari dataframe:

In [3]:
# 1) get kolom
tips[['total_bill', 'tip']].head()

,total_bill,tip
0,16.99,1.01
1,10.34,1.66
2,21.01,3.50
3,23.68,3.31
4,24.59,3.61


In [4]:
# 2) get beberapa baris
tips[3:5]

,total_bill,tip,sex,smoker,day,time,size
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [5]:
# 3) select baris and kolom berdasarkan nama
tips.loc[2:4, 'sex': 'smoker']

,sex,smoker
2,Male,No
3,Male,No
4,Female,No


In [6]:
# select baris and kolom berdasarkan urutannya
tips.iloc[1:3, 0:2]

,total_bill,tip
1,10.34,1.66
2,21.01,3.50


In [7]:
# 5) select menggunakan condition
tips[tips['tip'] > 1].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Multi-index

Judul yang mungkin tidak Anda butuhkan - tetapi ternyata menjadi kasus penggunaan yang agak sering.

Ide awal di balik multi-index adalah menyediakan framework untuk bekerja dengan data redup yang lebih tinggi (dan dengan demikian menjadi pengganti panel).

Mari kita lakukan contoh di bawah ini:

In [8]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [9]:
mi_tips = tips.groupby(['sex', 'smoker']).agg({'tip': 'mean'})
mi_tips

tip
sex    smoker          
Male   Yes     3.051167
       No      3.113402
Female Yes     2.931515
       No      2.773519

In [10]:
mi_tips.index

MultiIndex([(  'Male', 'Yes'),
            (  'Male',  'No'),
            ('Female', 'Yes'),
            ('Female',  'No')],
           names=['sex', 'smoker'])

Pada akhirnya, ada banyak sekali operasi yang dapat Anda lakukan di atas jenis data ini. Dan ada operasi multi-index setara yang dapat Anda lakukan, seperti ini:

In [11]:
mi_tips.loc[('Male', 'No')]

tip    3.113402
Name: (Male, No), dtype: float64

Tetapi dengan cara itu Anda akan mempelajari banyak detail dan selalu ada pengecualian.

Jadi cara yang bisa dilakukan hanyalah dengan mereset index.

In [12]:
ri_tips = mi_tips.reset_index()
ri_tips

,sex,smoker,tip
0,Male,Yes,3.051167
1,Male,No,3.113402
2,Female,Yes,2.931515
3,Female,No,2.773519


Perhatikan bagaimana kita mendapatkan nilai yang tersebar di seluruh kolom sekarang. Jadi dengan cara ini mudah untuk memilih pria bukan perokok:

In [13]:
ri_tips[(ri_tips['smoker'] == 'No') & (ri_tips['sex'] == 'Male')]

,sex,smoker,tip
1,Male,No,3.113402


Cara lain untuk mengatasi hal ini adalah dengan hanya mengeluarkan indeks tertentu:

In [14]:
ri0_tips = mi_tips.reset_index(level=0)
ri0_tips.loc['Yes']

,sex,tip
smoker,,
Yes,Male,3.051167
Yes,Female,2.931515


Dan akhirnya Anda dapat menarik kembali indeks ke dalam indeks (pada dasarnya hanya berguna untuk jenis penggabungan tertentu ).

In [15]:
ri_tips.set_index(['sex', 'smoker'])

tip
sex    smoker          
Male   Yes     3.051167
       No      3.113402
Female Yes     2.931515
       No      2.773519

In [16]:
ri0_tips.set_index('sex', append=True)

,,tip
smoker,sex,
Yes,Male,3.051167
No,Male,3.113402
Yes,Female,2.931515
No,Female,2.773519


# Getting Single Values

Trik pengindeksan kecil berikutnya adalah yang sebagian besar tentang kecepatan. Ini dapat digunakan ketika mendapatkan dan menetapkan nilai tunggal. Ini cukup sederhana:

In [17]:
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


Saat mendapatkan/mengatur nilai tunggal, Anda harus menggunakan fungsi `at`

In [18]:
tips.at[0, 'total_bill'] = 9000
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,9000.00,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [19]:
tips.iat[0, 0]

9000.0

If you are modifying single values of a dataframe you should always use these guys. It's faster and it is a good way to know that you are not messing up (often times modifying the data can result in odd errors).

So just to prove it's faster let's time it!

Jika Anda memodifikasi nilai tunggal dari dataframe, anda harus selalu menggunakan hal ini. Penggunaan 'at' lebih cepat. Mari kita buktikan hal tersebut

In [20]:
%%timeit
tips.at[0, 'total_bill'] = 6

8.33 µs ± 532 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
%%timeit
tips.loc['total_bill', 0] = 6

68.6 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Where, Masks and Queries

In [22]:
df = pd.DataFrame(np.random.randn(25).reshape((5, 5)))
df.head()

,0,1,2,3,4
0,-1.633994,0.766036,-0.213028,-1.583895,0.435069
1,-0.505325,0.157533,-0.174803,0.630488,-0.580454
2,-0.325903,-0.046123,-0.515029,0.144017,-1.101403
3,0.813104,-0.327957,-3.012866,0.391105,-1.663408
4,-0.365741,0.115183,-1.133716,0.071652,-0.382382


In [23]:
df.where(df > 0)

,0,1,2,3,4
0,NaN,0.766036,NaN,NaN,0.435069
1,NaN,0.157533,NaN,0.630488,NaN
2,NaN,NaN,NaN,0.144017,NaN
3,0.813104,NaN,NaN,0.391105,NaN
4,NaN,0.115183,NaN,0.071652,NaN


In [24]:
df[df < 0] = np.NaN
df

,0,1,2,3,4
0,NaN,0.766036,NaN,NaN,0.435069
1,NaN,0.157533,NaN,0.630488,NaN
2,NaN,NaN,NaN,0.144017,NaN
3,0.813104,NaN,NaN,0.391105,NaN
4,NaN,0.115183,NaN,0.071652,NaN
